In [ ]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
#G마켓 로그인 주소로 들어가기

driver = Chrome(executable_path='./Chromedriver')
url = 'https://signinssl.gmarket.co.kr/LogOut/LogOut'
driver.get(url)

In [ ]:
#지마켓 로그인
def SummitAccount() :
    AccountID = input("아이디를 입력하세요 : ")
    Password = input("비번을 입력하세요 :")
    IDform = driver.find_element(By.XPATH, '//*[@id="id"]')
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="id"]'))).send_keys(AccountID)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="pwd"]'))).send_keys(Password)
    driver.find_element(By.XPATH, '//*[@id="mem_login"]/div[1]/div[3]/button/span').click()
    return "로그인이 완료되었습니다."

In [ ]:
#지마켓 로그인 - 실행
SummitAccount()

In [ ]:
#장바구니로 이동후 파싱

soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
# 지마켓 각 정보를 빼내는 클래스
class Gmarket_Cart():
    Carts = soup.find_all("li", class_="cart--basket type_minishop") 
    IDList = []
    
    def __init__(self) :
        #실행시 일어나는 일
        print("실행 준비 끝")
        return
    
    
    def MakeArray(self) :
        '''표의 인덱스를 (장바구니, 장바구니 내부)의 형태의 array로 만들어줍니다.'''
        #나중에 멀티 인덱스로 묶어주기 위한 밑작업 - 인덱스 행렬 만들기
        BlankArray = []
        for i in range(0, len(self.Carts)):
            ItemNames = self.Carts[i].find_all("span", class_="item_name")
            for j in range(0, len(ItemNames)):
                Arrays = (i,j)
                BlankArray.append(Arrays)
        return BlankArray
                
    
    def ItemNames(self) :
        '''물품명을 찾아줍니다.'''
        #물품명을 딕셔너리로 만들어주기 (판다스에 열(col)로 넣을 것들)
        ItemNamesList = []
        for i in range(0, len(self.Carts)) :
            ItemNames = self.Carts[i].find_all("span", class_="item_name")
            
            for j in range(0, len(ItemNames)) :
                ItemNamesList.append(ItemNames[j].text)
        DictIN = {"물품명" : ItemNamesList}
        return DictIN
    
    
    
    def ItemPrices(self) :
        '''물품 가격을 찾아줍니다.'''
        #물품 가격을 딕셔너리로 만들어주기 (판다스에 열(col)로 넣을 것들)
        ItemPricesList = []

        for i in range(0, len(self.Carts)) :
            ItemPrices = self.Carts[i].find_all("div", class_="section item_price")
            
            for j in range(0, len(ItemPrices)) :
                ItemPricesList.append(int(ItemPrices[j].text[8:-1].replace(",","")))
        DictIP = {"물품 가격" : ItemPricesList}
        return DictIP
    
    
    def ItemQtys(self) :
        '''물품 개수를 찾아줍니다.'''
        #물품 개수를 딕셔너리로 만들어주기 (판다스에 열(col)로 넣을 것들)
        ItemQtysList = []

        for i in range(0, len(self.Carts)) :
            ItemQtys = self.Carts[i].find_all("input", class_="item_qty_count")
        
            for j in range(0, len(ItemQtys)) :
                ItemQtysList.append(ItemQtys[j]["value"])
        DictIQ = {"개수" : ItemQtysList}
        return DictIQ
    
    
    def ItemDLP(self) :
        '''택배비 값을 찾아내서 넣습니다.'''
        #각 카트에 들어간 배송비 계산
        ItemDLPList = []
        # 카트의 개수를 반복횟수로 삼아서
        for i in range(0, len(self.Carts)) :
            # 카트를 전부 리스트로 반환
            ItemDLP1 = self.Carts[i].find_all("div", class_="sub_sec delivery")
            # 카트안에 택배비를 꺼내야 한다.
            for j in range(0, len(ItemDLP1)) :
                ItemDLP2 = ItemDLP1[j].find("strong", class_="text__value")
                if bool(ItemDLP2) == True : 
                    ItemDLPList.append(int(ItemDLP2.text.replace(",",'')))
                else :
                    ItemDLPList.append(0)
        #택배비를 딕셔너리화
        DictIDLP = {"택배비" : ItemDLPList}
        return ItemDLPList
                
            
    
    def DF(self) :
        '''판다스 데이터 프레임을 만듭니다.'''
        a = self.ItemNames()
        b = self.ItemPrices()
        c = self.ItemQtys()
        d = self.ItemDLP()
        array = self.MakeArray()
        DictAll ={}
        DictAll.update(a)
        # 변수는 대충 a라도 둠, 귀찮아!
        DictAll.update(b)
        # 나중에 고쳐주자...
        DictAll.update(c)
        # 미안해 클린코드...
        df = pd.DataFrame(data=DictAll, index = pd.MultiIndex.from_tuples(array, names=['카트번호', '물품번호']))
        # 마지막에 멀티 인덱스 (i,0) 앞에 i만 변화하고 항상 0번째 인덱스에 택배비 포함 시켜줌
        for i in range(0, len(self.Carts)):
            df.loc[(i,0), "택배비"] = d[i]
        return df
    
    
    def SumPrice(self) :
        '''가격 총합을 계산합니다.'''
        #물품 가격 총합
        a = self.DF()
        SumPrices = int(a["물품 가격"].sum(0))
        return SumPrices
    
    def SumDLP(self) :
        '''택배비 총합을 계산합니다.'''
        #택배비 총합
        a = self.DF()
        SumDLPs = int(a["택배비"].sum(0))
        return SumDLPs
        
        

In [ ]:
CartRider = Gmarket_Cart()

In [ ]:
df = CartRider.DF()

In [ ]:
df.to_csv(".//test.csv", encoding='utf-8-sig')